In [4]:
import cv2
import os
import imutils
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

personName = 'Jair'
dataPath = 'Data' # Cambia a la ruta donde hayas almacenado Data
personPath = dataPath + '/' + personName

if not os.path.exists(personPath):
    print('Carpeta creada:', personPath)
    os.makedirs(personPath)

cap = cv2.VideoCapture('JAIR1.mp4')
print('Detectando')

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Crear el generador de imágenes aumentadas
datagen = ImageDataGenerator(
    rotation_range=20,  # Rango de rotación de las imágenes
    width_shift_range=0.1,  # Rango de desplazamiento horizontal
    height_shift_range=0.1,  # Rango de desplazamiento vertical
    shear_range=0.1,  # Rango de deformación de corte
    zoom_range=0.1,  # Rango de zoom
    horizontal_flip=True,  # Volteo horizontal aleatorio
    brightness_range=(0.9, 1.1)  # Rango de brillo
)

count = 0

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    frame = imutils.resize(frame, width=640)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = frame.copy()

    faces = faceClassif.detectMultiScale(gray, 1.2, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        rostro = auxFrame[y:y+h, x:x+w]
        rostro = cv2.resize(rostro, (150, 150), interpolation=cv2.INTER_CUBIC)

        # Generar imágenes aumentadas
        augmented_images = datagen.flow(np.array([rostro]), batch_size=1)

        for i, augmented_image in enumerate(augmented_images):
            if i >= 5:  # Guardar solo 5 imágenes aumentadas por cada detección
                break

            augmented_image = augmented_image[0].astype(np.uint8)

            # Girar imagen
            rotated_image = cv2.rotate(augmented_image, cv2.ROTATE_90_CLOCKWISE)

            # Voltear lateralmente
            flipped_image = cv2.flip(augmented_image, 1)

            # Aplicar zoom
            zoomed_image = cv2.resize(augmented_image, None, fx=1.2, fy=1.2)

            # Guardar imágenes aumentadas
            cv2.imwrite(personPath + '/rostro_{}.jpg'.format(count), augmented_image)
            count += 1
            cv2.imwrite(personPath + '/rostro_{}.jpg'.format(count), rotated_image)
            count += 1
            cv2.imwrite(personPath + '/rostro_{}.jpg'.format(count), flipped_image)
            count += 1
            cv2.imwrite(personPath + '/rostro_{}.jpg'.format(count), zoomed_image)
            count += 1

    cv2.imshow('frame', frame)

    k = cv2.waitKey(1)
    if k == 27 or count >= 2000:
        break

print('FIN')

cap.release()
cv2.destroyAllWindows()


Carpeta creada: Data/Messi
Detectando
FIN
